In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import urllib,time
# import tushare as ts

In [20]:
df = pd.read_csv("000001.csv", encoding ="ANSI")

In [21]:
df.head(5)

,date,股票代码,名称,close,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,成交量,成交金额
0,1990/12/19,'000001,上证指数,99.98,99.98,95.79,96.05,None,None,None,1260,494000
1,1990/12/20,'000001,上证指数,104.39,104.39,99.98,104.30,99.98,4.41,4.4109,197,84000
2,1990/12/21,'000001,上证指数,109.13,109.13,103.73,109.07,104.39,4.74,4.5407,28,16000
3,1990/12/24,'000001,上证指数,114.55,114.55,109.13,113.57,109.13,5.42,4.9666,32,31000
4,1990/12/25,'000001,上证指数,120.25,120.25,114.55,120.09,114.55,5.7,4.976,15,6000


In [22]:
df.tail(5)

,date,股票代码,名称,close,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,成交量,成交金额
6996,2019/7/31,'000001,上证指数,2932.5058,2944.3977,2926.4853,2944.3977,2952.3383,-19.8325,-0.6718,141052348,1.65E+11
6997,2019/8/1,'000001,上证指数,2908.7660,2927.3421,2901.7503,2920.8506,2932.5058,-23.7398,-0.8095,142389813,1.71E+11
6998,2019/8/2,'000001,上证指数,2867.8376,2872.4730,2851.4402,2861.3323,2908.766,-40.9284,-1.4071,184888443,2.16E+11
6999,2019/8/5,'000001,上证指数,2821.4957,2863.6891,2821.4957,2854.5780,2867.8376,-46.3419,-1.6159,167419423,1.97E+11
7000,2019/8/6,'000001,上证指数,2777.5559,2787.4185,2733.9242,2776.9889,2821.4957,-43.9398,-1.5573,231123727,2.55E+11


# 1. 极端值涨跌幅预测因子

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  6 14:31:49 2019

@author: hurenjie
"""

# 极端值因子：当大盘(或者其他股票)日涨幅/跌幅绝对值超过2%，下一天上涨或者下跌的概率/历史上的次数有多少
#%%

# token='2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67'
# pro = ts.pro_api(token)
# df = ts.get_hist_data('sh').sort_index()

#%%
def extreme_value(df, threshold): # threshold 在此应该定义为一个百分数,代表我们认为的涨跌幅度阈值
    df['close_pct_ch'] = df['close'].pct_change().shift(-1)
    df['close_diff'] = df['close'].diff().shift(-2)
    df_up = df[df['close_diff'] > 0]
    df_down = df[df['close_diff'] < 0]
    
    up = df_up[df_up['close_pct_ch'] > threshold]
    down = df_down[df_down['close_pct_ch'] < -threshold]
    
    prob_up = np.shape(up)[0]/np.shape(df_up)[0]
    prob_down = np.shape(down)[0]/np.shape(df_down)[0]
        
    if df['close_diff'].iloc[-3] > 0:
        result = prob_up # 当天价格上涨，利用极端值因子估计法估计出第二天继续上涨的概率
        print('当天价格上涨,该函数计算的是第二天价格继续上涨的概率')
    else:
        result = prob_down # 当天价格下跌，利用极端值因子估计法估计出第二天继续下跌的概率
        print('当天价格下跌,该函数计算的事第二天价格继续下跌的概率')
    df.drop(columns = ['close_pct_ch', 'close_diff'], inplace = True)
    return result

In [10]:
extreme_value(df, 0.0156)

当天价格下跌,该函数计算的事第二天价格继续下跌的概率


0.12265861027190332

In [16]:
# threshold = 0.0156
# df['close_pct_ch'] = df['close'].pct_change().shift(-1)
# df['close_diff'] = df['close'].diff().shift(-2)
# df_up = df[df['close_diff'] > 0]
# df_down = df[df['close_diff'] < 0]

# up = df_up[df_up['close_pct_ch'] > threshold]
# down = df_down[df_down['close_pct_ch'] < -threshold]

# prob_up = np.shape(up)[0]/np.shape(df_up)[0]
# prob_down = np.shape(down)[0]/np.shape(df_down)[0]

# if df['close_diff'].iloc[-3] > 0:
#     result = prob_up # 当天价格上涨，利用极端值因子估计法估计出第二天继续上涨的概率
#     print('当天价格上涨,该函数计算的是第二天价格继续上涨的概率')
# else:
#     result = prob_down # 当天价格下跌，利用极端值因子估计法估计出第二天继续下跌的概率
#     print('当天价格下跌,该函数计算的事第二天价格继续下跌的概率')
# # df.drop(columns = ['close_pct_ch', 'close_diff'], inplace = True)

# 2. 简单连续涨跌预测因子

In [96]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Aug  6 16:15:22 2019

@author: hurenjie
"""
# 简单连续涨跌因子：连续涨/跌N（3、4、5、6）天之后，下一天涨/跌的概率
#%% 
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
# import tushare as ts

#token='2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67'
#pro = ts.pro_api(token)
#df = ts.get_hist_data('sh').sort_index()

#%%
def simple_predict_up_or_down(df, N): # N代表我们认为的连续多少天持续的涨或者跌
    df.reset_index(inplace = True)
    df['close_diff'] = df['close'].diff()
    df['signal'] = 0
    for i in df.index:
        if df['close_diff'][i] < 0:
            df['signal'][i] = -1
        elif df['close_diff'][i] > 0:
            df['signal'][i] = 1
    
    df['signal_sum'] = df['signal'].rolling(N).sum()
    df_up_Ndays = df[df['signal_sum'] == N]
    next_day_is_up = 0
    
    df_down_Ndays = df[df['signal_sum'] == -N]
    next_day_is_down = 0
    
    for i in df_up_Ndays.index:
        if i!=df.index[-1]:
            if df['close_diff'][i+1] > 0:
                next_day_is_up += 1
    
    for i in df_down_Ndays.index:
        if i!=df.index[-1]:
            if df['close_diff'][i+1] < 0:
                next_day_is_down += 1
    df.drop(columns = ['close_diff', 'signal', 'signal_sum'], inplace = True)
    df.set_index('date', inplace = True)
    
    prob_up = next_day_is_up/np.shape(df_up_Ndays)[0]
    prob_down = next_day_is_down/np.shape(df_down_Ndays)[0]
    
    return {'连续'+ str(N) +'天上涨后第二天上涨的概率': prob_up, '连续'+ str(N) +'天下跌后第二天下跌的概率':prob_down}

# 网易爬数据

In [25]:
def get_page(url):  #获取页面数据
    req=urllib.request.Request(url,headers={
        'Connection': 'Keep-Alive',
        'Accept': 'text/html, application/xhtml+xml, */*',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko'
    })
    opener=urllib.request.urlopen(req)
    page=opener.read()
    return page

def get_index_history_byNetease(index_temp):
    """
    :param index_temp: for example, 'sh000001' 上证指数
    :return:
    """
    index_type=index_temp[0:2]
    index_id=index_temp[2:]
    if index_type=='sh':
        index_id='0'+index_id
    if index_type=="sz":
        index_id='1'+index_id
    url='http://quotes.money.163.com/service/chddata.html?code=%s&start=19900101&end=%s&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;VOTURNOVER;VATURNOVER'%(index_id,time.strftime("%Y%m%d"))

    page=get_page(url).decode('gb2312') #该段获取原始数据
    page=page.split('\r\n')
    col_info=page[0].split(',')   #各列的含义
    index_data=page[1:]     #真正的数据

    #为了与现有的数据库对应，这里我还修改了列名，大家不改也没关系
    col_info[col_info.index('日期')]='交易日期'   #该段更改列名称
    col_info[col_info.index('股票代码')]='指数代码'
    col_info[col_info.index('名称')]='指数名称'
    col_info[col_info.index('成交金额')]='成交额'

    index_data=[x.replace("'",'') for x in index_data]  #去掉指数编号前的“'”
    index_data=[x.split(',') for x in index_data]

    index_data=index_data[0:index_data.__len__()-1]   #最后一行为空，需要去掉
    pos1=col_info.index('涨跌幅')
    pos2=col_info.index('涨跌额')
    posclose=col_info.index('收盘价')
    index_data[index_data.__len__()-1][pos1]=0      #最下面行涨跌额和涨跌幅为None改为0
    index_data[index_data.__len__()-1][pos2]=0
    for i in range(0,index_data.__len__()-1):       #这两列中有些值莫名其妙为None 现在补全
        if index_data[i][pos2]=='None':
            index_data[i][pos2]=float(index_data[i][posclose])-float(index_data[i+1][posclose])
        if index_data[i][pos1]=='None':
            index_data[i][pos1]=(float(index_data[i][posclose])-float(index_data[i+1][posclose]))/float(index_data[i+1][posclose])

    # print(col_info)
    return [index_data,col_info]
# --------------------- 
# 版权声明：本文为CSDN博主「multiangle」的原创文章，遵循CC 4.0 by-sa版权协议，转载请附上原文出处链接及本声明。
# 原文链接：https://blog.csdn.net/u014595019/article/details/48445223

In [26]:
sh = get_index_history_byNetease("sh000001") 

In [31]:
sh[0]

[['2019-08-06',
  '000001',
  '上证指数',
  '2777.5559',
  '2787.4185',
  '2733.9242',
  '2776.9889',
  '2821.4957',
  '-43.9398',
  '-1.5573',
  '231123727',
  '2.55209231213e+11'],
 ['2019-08-05',
  '000001',
  '上证指数',
  '2821.4957',
  '2863.6891',
  '2821.4957',
  '2854.578',
  '2867.8376',
  '-46.3419',
  '-1.6159',
  '167419423',
  '1.96891483092e+11'],
 ['2019-08-02',
  '000001',
  '上证指数',
  '2867.8376',
  '2872.473',
  '2851.4402',
  '2861.3323',
  '2908.766',
  '-40.9284',
  '-1.4071',
  '184888443',
  '2.16327400623e+11'],
 ['2019-08-01',
  '000001',
  '上证指数',
  '2908.766',
  '2927.3421',
  '2901.7503',
  '2920.8506',
  '2932.5058',
  '-23.7398',
  '-0.8095',
  '142389813',
  '1.71115635675e+11'],
 ['2019-07-31',
  '000001',
  '上证指数',
  '2932.5058',
  '2944.3977',
  '2926.4853',
  '2944.3977',
  '2952.3383',
  '-19.8325',
  '-0.6718',
  '141052348',
  '1.64607873474e+11'],
 ['2019-07-30',
  '000001',
  '上证指数',
  '2952.3383',
  '2965.632',
  '2946.2566',
  '2946.2566',
  '2941.007'

In [69]:
df_ne = pd.DataFrame()

In [70]:
df_ne

""


In [71]:
df_ne['columns_to_split'] = pd.DataFrame(sh).T[0]

In [72]:
col_list = list(pd.DataFrame(sh).T[1])[:12]

In [73]:
col_list

['交易日期',
 '指数代码',
 '指数名称',
 '收盘价',
 '最高价',
 '最低价',
 '开盘价',
 '前收盘',
 '涨跌额',
 '涨跌幅',
 '成交量',
 '成交额']

In [74]:
df_ne['columns_to_split'] = df_ne['columns_to_split'].apply(lambda x: str(x))
df_ne['columns_to_split'] = df_ne['columns_to_split'].apply(lambda x: x.replace("[", ""))
df_ne['columns_to_split'] = df_ne['columns_to_split'].apply(lambda x: x.replace("]", ""))

In [75]:
df_ne

,columns_to_split
0,"'2019-08-06', '000001', '上证指数', '2777.5559', '..."
1,"'2019-08-05', '000001', '上证指数', '2821.4957', '..."
2,"'2019-08-02', '000001', '上证指数', '2867.8376', '..."
3,"'2019-08-01', '000001', '上证指数', '2908.766', '2..."
4,"'2019-07-31', '000001', '上证指数', '2932.5058', '..."
5,"'2019-07-30', '000001', '上证指数', '2952.3383', '..."
6,"'2019-07-29', '000001', '上证指数', '2941.007', '2..."
7,"'2019-07-26', '000001', '上证指数', '2944.541', '2..."
8,"'2019-07-25', '000001', '上证指数', '2937.3601', '..."
9,"'2019-07-24', '000001', '上证指数', '2923.2775', '..."


In [84]:
df_ne.head(2)

,columns_to_split
0,"'2019-08-06', '000001', '上证指数', '2777.5559', '..."
1,"'2019-08-05', '000001', '上证指数', '2821.4957', '..."


In [90]:
df_ne = df_ne['columns_to_split'].str.split(",", 12, expand = True)

In [91]:
df_ne.columns = col_list

In [95]:
df_ne.sort_values("交易日期").tail()

,交易日期,指数代码,指数名称,收盘价,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,成交量,成交额
4,'2019-07-31','000001','上证指数','2932.5058','2944.3977','2926.4853','2944.3977','2952.3383','-19.8325','-0.6718','141052348','1.64607873474e+11'
3,'2019-08-01','000001','上证指数','2908.766','2927.3421','2901.7503','2920.8506','2932.5058','-23.7398','-0.8095','142389813','1.71115635675e+11'
2,'2019-08-02','000001','上证指数','2867.8376','2872.473','2851.4402','2861.3323','2908.766','-40.9284','-1.4071','184888443','2.16327400623e+11'
1,'2019-08-05','000001','上证指数','2821.4957','2863.6891','2821.4957','2854.578','2867.8376','-46.3419','-1.6159','167419423','1.96891483092e+11'
0,'2019-08-06','000001','上证指数','2777.5559','2787.4185','2733.9242','2776.9889','2821.4957','-43.9398','-1.5573','231123727','2.55209231213e+11'


In [93]:
df.tail(5)

,date,股票代码,名称,close,最高价,最低价,开盘价,前收盘,涨跌额,涨跌幅,成交量,成交金额
6996,2019/7/31,'000001,上证指数,2932.5058,2944.3977,2926.4853,2944.3977,2952.3383,-19.8325,-0.6718,141052348,1.65E+11
6997,2019/8/1,'000001,上证指数,2908.7660,2927.3421,2901.7503,2920.8506,2932.5058,-23.7398,-0.8095,142389813,1.71E+11
6998,2019/8/2,'000001,上证指数,2867.8376,2872.4730,2851.4402,2861.3323,2908.766,-40.9284,-1.4071,184888443,2.16E+11
6999,2019/8/5,'000001,上证指数,2821.4957,2863.6891,2821.4957,2854.5780,2867.8376,-46.3419,-1.6159,167419423,1.97E+11
7000,2019/8/6,'000001,上证指数,2777.5559,2787.4185,2733.9242,2776.9889,2821.4957,-43.9398,-1.5573,231123727,2.55E+11
